In [ ]:
from datasets import wine_red_dataset

wine_red = wine_red_dataset()
wine_red.dropna(how='all')

# Feature selection

When a model is trained with colinear features, the coefficients (weights) tend to depend on each other which reduces explanability of the model. Find features that have high colinearity. 

Variance Inflation Factor, measures how much the variance of an estimated regression coefficient is increased because of collinearity

# Instance selection

# Correlating Features

# Feature reduction